In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import imageio
import cv2
import io
import math
from data_gen import rotation

from scipy.interpolate import interp1d

sys.path.append('../')

from utils import config
from utils import utils

In [2]:
def upsample_skeleton(skel, imu):
    """
    skeleton: (3, joints=20, N)
    imu: (3, M)
    """
    skel_measurements = skel.shape[2] # N
    imu_measurements = imu.shape[1] # M
    
    inv_sampling_rate = 1 / config.mhad_accel_sampling_rate
    time_window = imu_measurements * inv_sampling_rate

    skel_ts = np.linspace(0, time_window, skel_measurements) # time stamps for the skeleton data
    imu_ts = np.linspace(0, time_window, imu_measurements) # time stamps for the imu data

    skel_ts = np.round(skel_ts, 2)
    imu_ts = np.round(imu_ts, 2)
    
    skel_upsample = np.empty((skel.shape[0], skel.shape[1], imu_ts.shape[0])) # (3, joints, M)
    for axis in range(skel.shape[0]):
        for joint in range(skel.shape[1]):
            f = interp1d(skel_ts, skel[axis, joint, :], kind='cubic')
            skel_upsample[axis, joint, :] = f(imu_ts)
    
    return skel_upsample

In [3]:
#region Plotting
def plot_acceel(accel):
    fig, ax = plt.subplots(3, 1)
    fig.subplots_adjust(hspace=0.5)
    
    ax[0].plot(accel[0, :], label="x")
    ax[0].legend()

    ax[1].plot(accel[1, :], label="y")
    ax[1].legend()

    ax[2].plot(accel[2, :], label="z")
    ax[2].legend()

    plt.show()

def plot_joint(skel, joint):
    fig, ax = plt.subplots(3, 1)
    fig.subplots_adjust(hspace=0.5)
    ax[0].plot(skel[0, joint, :], label="x")
    ax[0].legend()
    ax[1].plot(skel[1, joint, :], label="y")
    ax[1].legend()
    ax[2].plot(skel[2, joint, :], label="z")
    ax[2].legend()

    plt.show()

def plot_sidebyside(acc, pose, joint):
    fig, ax = plt.subplots(3, 2)
    fig.subplots_adjust(hspace=0.5)
    ax[0][0].plot(acc[0, :], label="x")
    ax[1][0].plot(acc[1, :], label="y")
    ax[2][0].plot(acc[2, :], label="z")

    ax[0][1].plot(pose[0, joint, :], label="x")
    ax[1][1].plot(pose[1, joint, :], label="y")
    ax[2][1].plot(pose[2, joint, :], label="z")

def get_img_from_fig(fig, dpi=120):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=dpi, bbox_inches="tight", pad_inches=0)
    buf.seek(0)
    img_arr = np.frombuffer(buf.getvalue(), dtype=np.uint8)
    buf.close()
    img = cv2.imdecode(img_arr, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGBA)
    return img

def plot_skeleton(skel, frame=None):
    '''
    skel: (3, joint, N)
    '''
    motion = skel.transpose(1, 0, 2)  # (17, 3, N)
    vlen = motion.shape[-1]
    
    joint_pairs = [[0, 1], [1, 2], [2, 3], [1, 4], [4, 5], [5, 6], [6, 7], [1, 8], [8, 9], [9, 10], [10, 11],
                   [3, 12], [12, 13], [13, 14], [14, 15], [3, 16], [16, 17], [17, 18], [18, 19]]
    joint_pairs_left = [[1, 4], [4, 5], [5, 6], [6, 7],
                        [3, 12], [12, 13], [13, 14], [14, 15]]
    joint_pairs_right = [[1, 8], [8, 9], [9, 10], [10, 11],
        [3, 16], [16, 17], [17, 18], [18, 19]]

    color_mid = "#00457E"
    color_left = "#02315E"
    color_right = "#2F70AF"

    for f in range(vlen) if frame is None else [frame]:
        j3d = motion[:, :, f]
        fig = plt.figure(figsize=(10, 10))
        ax = plt.axes(projection="3d")
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.view_init(elev=180., azim=90)

        ax.scatter(0, 0, 0, color='red', s=100)  # 's' is the size of the point


        plt.tick_params(left=False, right=False, labelleft=False,
                        labelbottom=False, bottom=False)

        for i in range(len(joint_pairs)):
            limb = joint_pairs[i]
            xs, ys, zs = [np.array([j3d[limb[0], j], j3d[limb[1], j]])
                        for j in range(3)]
            if joint_pairs[i] in joint_pairs_left:
                ax.plot(-xs, -ys, -zs, color=color_left, lw=3, marker='o', markerfacecolor='w',
                        markersize=3, markeredgewidth=2)  # axis transformation for visualization
            elif joint_pairs[i] in joint_pairs_right:
                ax.plot(-xs, -ys, -zs, color=color_right, lw=3, marker='o', markerfacecolor='w',
                        markersize=3, markeredgewidth=2)  # axis transformation for visualization
            else:
                ax.plot(-xs, -ys, -zs, color=color_mid, lw=3, marker='o', markerfacecolor='w',
                        markersize=3, markeredgewidth=2)  # axis transformation for visualization

            for j in range(j3d.shape[0]):
                x, y, z = j3d[j, 0], j3d[j, 1], j3d[j, 2]
                ax.text(-x, -y, -z, j, color='k', fontsize=10, weight='bold')

        fig.savefig(f'tmp/debug_frame_{f}.png')
        # frame_vis = get_img_from_fig(fig)
        # cv2.imwrite(f'tmp/debug_frame_{f}.png', frame_vis)  # Save the image to the provided path
        if frame is not None:
            plt.show()
        else:
            plt.close(fig)
#endregion

In [4]:
file_pairs = {} # dictionary to store pairs of [skeleton and inertial] data for each [action_subject_trial]
for s in config.MHAD_SUBJECTS:
    s_dir = os.path.join(config.mhad_data_dir, s)

    files = sorted(os.listdir(s_dir))
    # group modalities
    for file in files:
        prefix = "_".join(file.split('_')[:3]) # pattern is 'a_s_t' where a is action, s is subject, t is trial.
        if prefix not in file_pairs:
            file_pairs[prefix] = {'inertial': None, 'skeleton': None}
        
        suffix = file.split('_')[-1]  # This will be either 'inertial' or 'skeleton'
        modality_type = suffix.split('.')[0]
       
        if modality_type == 'inertial' or modality_type == 'skeleton':
            modality = np.load(os.path.join(s_dir, file)) # load modality. intertial: (N, 6), skeleton: (joints=20, 3, N)
            if modality_type == 'inertial':
                modality = modality[:, :3] # only accelerometer. drop gyroscope. (N, 3)
                modality = np.transpose(modality, (1, 0)) # convert to (3, N)
            elif modality_type == 'skeleton':
                modality = np.transpose(modality, (1, 0, 2)) # convert to (3, joints=20, N)
        
            file_pairs[prefix][modality_type] = modality # skeleton or inertial

        else: # inertial_std or skeleton_upsample_normal
            continue

In [5]:
# >>> Step 1: Upsample Skeleton data <<< #
for pair in file_pairs.values():
    imu = pair['inertial']
    skel = pair['skeleton']

    skel_upsample = upsample_skeleton(skel=skel, imu=imu)
    pair['skeleton'] = skel_upsample

In [6]:
# >>> Step 2: Standardaize IMU <<< #
train_accel = []

for prefix, pair in file_pairs.items():
    contains = any(s in prefix for s in config.MHAD_TRAIN_S_IDS)
    if contains:
        imu = pair['inertial']
        train_accel.append(imu)

train_accel = np.concatenate(train_accel, axis=1) # shape (3, N)

total_mean = np.mean(train_accel, axis=1) # shape (3,)
total_std = np.std(train_accel, axis=1) # shape (3,)

for prefix, pair in file_pairs.items():
    imu = pair['inertial']
    for i in range(3):
        imu[i, :] = (imu[i, :] - total_mean[i]) / (total_std[i] + 1e-8)
    
    pair['inertial'] = imu
    
    # save the standardized data
    s = prefix.split('_')[1]
    s_dir = os.path.join(config.mhad_data_dir, s)
    np.save(os.path.join(s_dir, prefix + '_' + config.mhad_inertial_file), imu)

In [7]:
def pad_null_frames(skeleton):
    skeleton = skeleton.transpose(2, 1, 0)  # to (N, 20, 3)
    for i_f, frame in enumerate(skeleton):
        if frame.sum() == 0:
            if skeleton[i_f:].sum() == 0:
                rest = len(skeleton) - i_f
                num = int(np.ceil(rest / i_f))
                pad = np.concatenate(
                    [skeleton[0:i_f] for _ in range(num)], 0)[:rest]
                skeleton[i_f:] = pad
                break
    
    return skeleton.transpose(2, 1, 0)

In [8]:
# >>> Step 3: Normalize Skeleton data <<< #
for prefix, pair in file_pairs.items():
    # (3, joints=20, N)
    skel = pair['skeleton']

    # Step 1: Extract MidHip and Neck coordinates
    midhip_coords = skel[:, 3, :]  # mid hip
    neck_coords = skel[:, 0, :]  # head

    # # Step 2: center skeleton data
    skel = skel - midhip_coords[:, np.newaxis, :]
    
    # Step 5: Calculate Euclidean distances for each sample N
    distances = np.linalg.norm(neck_coords - midhip_coords, axis=0)

    # Step 4: Calculate median distance using a sliding window of 2 seconds
    median_distances = np.array(
        [np.median(distances[max(0, t-50):min(skel.shape[2], t+51)]) for t in range(skel.shape[2])])

    # Step 5: Normalize all joints coordinates
    skel_normal = (skel / median_distances)

    skel_normal = skel_normal[[0, 2, 1], :, :]  # (3, 20, N) -> (3, 20, N) x, z, y
    
    pair['skeleton'] = skel_normal
    
    # save the normalized data
    s = prefix.split('_')[1]
    s_dir = os.path.join(config.mhad_data_dir, s)
    np.save(os.path.join(s_dir, prefix + '_' + config.mhad_skeleton_file), skel_normal)

# 0. head; 
# 1. shoulder_center;
# 2. spine;
# 3. hip_center;
# 4. left_shoulder;
# 5. left_elbow;
# 6. left_wrist;
# 7. left_hand;
# 8. right_shoulder;
# 9. right_elbow;
# 10. right_wrist;
# 11. right_hand;
# 12. left_hip;
# 13. left_knee;
# 14. left_ankle;
# 15. left_foot;
# 16. right_hip;
# 17. right_knee;
# 18. right_ankle;
# 19. right_foot;

In [9]:
name = 'a1_s1_t1'
skel = file_pairs[name]['skeleton']
# p_skel = np.zeros((3, 20, 300), dtype=np.float32)
# p_skel[:, :, :skel.shape[2]] = skel
# p_skel = pad_null_frames(p_skel)
# plot_skeleton(skel)
# plot_joint(skel, 8)